In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [2]:
# 定义一个工具
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [3]:
tools = [get_word_length]

#由于 OpenAI 函数调用针对工具使用进行了微调，我们几乎不需要任何关于如何推理或如何输出格式的说明。我们只有两个输入变量：input和agent_scratchpad。input应该是包含用户目标的字符串。agent_scratchpad应该是包含先前代理工具调用和相应工具输出的消息序列。


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [7]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_T8BCBubfP6cVdPhGsW03k5qK', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7'}, id='run-d7794aee-eb8b-419c-a59b-80eb2ebfbf2e', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_T8BCBubfP6cVdPhGsW03k5qK', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_T8BCBubfP6cVdPhGsW03k5qK', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_T8BCBubfP6cVdPhGsW03k5qK')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_T8BCBubfP6cVdPhGsW03k5qK'

In [9]:
#如果我们将其与基础 LLM 进行比较，我们可以看到 LLM 本身就很吃力

llm.invoke("How many letters in the word educa")

AIMessage(content='The word "educa" has 5 letters.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7', 'finish_reason': 'stop', 'logprobs': None}, id='run-8323eb23-cac7-4fc4-8cce-413f2bc01cdc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26})

In [10]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [12]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
"Educa" is not a standard English word; it may be a variation of "educate" or could be used in specific contexts, such as names or brands. If you need more information or a specific context, feel free to ask!

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': '"Educa" is not a standard English word; it may be a variation of "educate" or could be used in specific contexts, such as names or brands. If you need more information or a specific context, feel free to ask!'}